# Fetching World Bank data using an API

An API is a communication line between your computer and a database owned by someone else. By accessing the API with Python, we can automate the process of downloading data. 

This notebook demonstrates how to Python to interact with the World Bank's API to download some data on life expectancy across countries. We will do this in two ways:
* Simple way: Using the pandas-datareader package.
* General way: Interacting directly with the API using the requests package.

## Simple way: Using **pandas-datareader**

The python package pandas-datareader is a package that helps you interact with a number of different API's. The package conects to the API, fetches the data and parses it in a way that makes it manageable for you to work with. As we will see later, this is a big help!

Before we download data, we should **always check out the documentation**! This helps us understand the syntax to use, which keywords to provide etc. The documentation for pandas-datareader is [here](https://pandas-datareader.readthedocs.io/en/latest/). We are interested in data from The World Bank, which is [here](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#remote-data-wb).

We should of course also know the **data source** well. Let's have a look at the World Bank's data page to find the name of the dataset we want.

In [ ]:
from pandas_datareader import wb ## wb is the World Bank API
import pandas as pd

The documentation tells us that we can look though the variables available with wb.get_indicators():

In [ ]:
indicators = wb.get_indicators()
display(indicators)

We can also search through indicators using wb.search():

In [ ]:
matches = wb.search('life expectancy at birth')
display(matches)

It seems like the indicator SP.DYN.LE00.IN contains the information we want. Let's set up the parameters for the API call and download the data:

In [ ]:
params = {'symbols': 'SP.DYN.LE00.IN',
          'countries': ['US', 'CA', 'MX'], ## US, Canada, Mexico
          'start': 1980,
          'end': 2022,} 
wbdf = wb.WorldBankReader(**params).read()

Let's checkout the data!

In [ ]:
wbdf.head()

In [ ]:
(wbdf.reset_index()
    .pivot(index='year', columns='country', values='SP.DYN.LE00.IN')
    .plot(title='Life Expectancy at Birth'))

## General way: Using the **requests** package

Some (most) API's don't have pre-written python packages that helps us interact with them. Instead, we must use the 

The package **requests** allows us to access the internet and download data. It is well suited for API access and web scraping.

In [ ]:
import requests

Let's get the same data directly from the world Bank API. See documentation [here](https://datahelpdesk.worldbank.org/knowledgebase/topics/125589-developer-information).

We access the API through an url. The documentation tells us that the "base" url for the World Bank API is "https://api.worldbank.org/v2". On top of that, we need to provide it with information about wich indicators, countries and years, we want:

In [ ]:
url = 'https://api.worldbank.org/v2'        ## base
url = url + '/country/USA;CAN;MEX'          ## countries
url = url + '/indicator/SP.DYN.LE00.IN'     ## indicator
url = url + '?date=1980:2022'               ## date range

url = url + '&format=json'                  ## response format
url = url + '&per_page=1000'                ## max per page

print(url)

Let's download the data using requests.get:

In [ ]:
wb_raw = requests.get(url).json() ## get the data and parse is as JSON

In [ ]:
## Let's have a look..
print(wb_raw)

## oh no...

We need to have a inspect the output a bit to see what is what... Turns out that wb_raw is a list, where the first element wb_raw[0] contains metadata about the API call and the second element wb_raw[1] contains the actual data. Let's clean it up a bit...

In [ ]:
wb_clean = (pd.DataFrame(wb_raw[1])
          .loc[:, ['countryiso3code', 'date', 'value']]
          .rename(columns={'countryiso3code': 'country',
                           'date': 'year',
                           'value': 'life_expectancy'})
          .replace({'country': {'CAN': 'Canada', 'MEX': 'Mexico', 'USA': 'United States'}})
          .pivot(index='year', columns='country', values='life_expectancy'))

In [ ]:
wb_clean.plot(title='Life Expectancy at Birth');

The data is the same but it took a few more steps to get to the end result - in the previous section, pandas-datareader constructed the url and did the data cleaning for us!

But it is nice to know how to do these things ourselves. We might one day need to work with an API that does not have a pre-written python package!

## Relevant info for working with API's
* Some API's are free to use. Some require you to authenticate yourself by providing a use-specific access token in the url. 
* API's sometimes limit the amount of data you can download in one request. If you want to download large amounts of data, you might need to split the download into several requests.
* Somtimes, API's also limit the number of requests you can make within a certain timespan. Remember that each time you run the line of code that downloads data (eg. wb.WorldBankReader.read, requests.get), you make a new request.
* Some API's require payment to use.

All of this differs from API to API. Therefore, **read the documentation**! It'll tell you exactly how to interact with the API and what information to provide it.